In [1]:
import cPickle
import os.path

api_key = cPickle.load( file( os.path.expanduser( '~/mediacloud_api_key.pickle' ), 'r' ) )

In [2]:
import cPickle
import os.path

cPickle.dump( api_key, file( os.path.expanduser( '~/mediacloud_api_key.pickle' ), 'wb' ) )

In [3]:
import mediacloud, json

mc = mediacloud.api.AdminMediaCloud( api_key)


In [4]:
def corenlp_annotated( s ):
    corenlp = s['corenlp']
    
    if 'annotated' not in corenlp:
        return True
    
    if corenlp['annotated'] == 'false':
        return False
    
    return True

In [5]:
#import cliff
from mediameter.cliff import Cliff

cliff_api = Cliff('http://civicdev.media.mit.edu', 8080)

# Flags

In [6]:
reload_stories = True
reprocess_with_cliff = True


In [11]:
controversy_stories = []

fetch_size = 100

if reload_stories:
    
    last_processed_stories_id = 0

    controversy_stories_shelve = shelve.open((os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories_shelve.shelve' ) ))
    
    while True:
        print "fetching with last_processed_stories_id", last_processed_stories_id
        stories = mc.storyList( solr_query='{~ controversy:1113 }', rows=fetch_size, corenlp=True, 
                               last_processed_stories_id=last_processed_stories_id)

        if len( stories ) == 0:
            break
            
        for s in stories:
            
            del s['description']

            if corenlp_annotated( s ):
                try:
                    s['cliff_results'] = cliff_api.parseNlpJson( s['corenlp'])
                except:
                    print 'Error on ', s['stories_id']                                      
            
                if 'cliff_results' not in s:
                    print 'skipping non cliff_results story', s['stories_id']
                else:

                    results = s['cliff_results']['results']

                    if 'people' in results:
                        people = results['people']
                        names = [ p['name'] for p in people ]

                        s[ 'names' ] = names

                        del s['cliff_results' ]

            else:  # not corenlp_annotated
                
                print s['stories_id'], 'not annotated'
                
            del s['corenlp']

        last_processed_stories_id = stories[-1]['processed_stories_id']
        
        #if len( controversy_stories_shelve.keys()) > 200:
        #    break

        controversy_stories.extend( stories)
        
        #if len( controversy_stories ) > 30:
        #    break

    stories_ids = sorted( controversy_stories_shelve.keys() )

    non_corenlp_annotated_stories = []
    cliff_error_stories = []
    cliff_no_result_stories = []
    cliff_no_people_stories = []

    for stories_id in stories_ids:
        s = controversy_stories_shelve[ stories_id]

        cliff_info = {}

        if corenlp_annotated( s ):
            try:
                cliff_info['cliff_results'] = cliff_api.parseNlpJson( s['corenlp'])
            except:
                print 'Cliff Error on ', stories_id

                cliff_error_stories.append( stories_id )

                continue

            if 'results' not in cliff_info['cliff_results']:
                cliff_no_result_stories.append( stories_id )

            else:

                results = cliff_info['cliff_results']['results']

                if 'people' in results:
                    people = results['people']
                    names = [ p['name'] for p in people ]

                    cliff_info[ 'names'] = names

                else:
                    cliff_no_people_stories.append( stories_id )

            controversy_stories_corenlp[ stories_id ] = cliff_info

        else:  # not corenlp_annotated

            print stories_id, 'not annotated'

            non_corenlp_annotated_stories.append( stories_id )

    cliff_problems = shelve.open(os.path.expanduser( '~/Dropbox/gamergate_data/cliff_problmes.shelve' ) , flag='n' )

    cliff_problems[ 'cliff_error_stories'] = cliff_error_stories
    cliff_problems[ 'cliff_no_result_stories'] = cliff_no_result_stories
    cliff_problems[ 'cliff_no_people_stories'] = cliff_no_people_stories
    cliff_problems[ 'non_corenlp_annotated_stories'] = non_corenlp_annotated_stories
    cliff_problems.close()
    controversy_stories_corenlp.close()
    controversy_stories_shelve.close()

fetching with last_processed_stories_id 0
124099751 not annotated
164918000 not annotated
174255563 not annotated
217751583 not annotated
219285013 not annotated
fetching with last_processed_stories_id 243575826
229263527 not annotated
237628466 not annotated
238668953 not annotated
239207216 not annotated
fetching with last_processed_stories_id 254719368
241748601 not annotated
fetching with last_processed_stories_id 256674847
Error on  244729537
skipping non cliff_results story 244729537
244742084 not annotated
fetching with last_processed_stories_id 259259277
245244212 not annotated
245354910 not annotated
fetching with last_processed_stories_id 261458055
Error on  248559106
skipping non cliff_results story 248559106
248628727 not annotated
Error on  249174413
skipping non cliff_results story 249174413
fetching with last_processed_stories_id 263897100
250790429 not annotated
250789636 not annotated
250795713 not annotated
242912598 not annotated
fetching with last_processed_stories_

ERROR:mediameter.cliff:('Connection aborted.', gaierror(-2, 'Name or service not known'))
Traceback (most recent call last):
  File "/usr/local/lib/python2.7/dist-packages/mediameter/cliff.py", line 49, in _query
    r = requests.post( self._urlTo(path), data=args)
  File "/usr/local/lib/python2.7/dist-packages/requests/api.py", line 109, in post
    return request('post', url, data=data, json=json, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/requests/api.py", line 50, in request
    response = session.request(method=method, url=url, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/requests/sessions.py", line 465, in request
    resp = self.send(prep, **send_kwargs)
  File "/usr/local/lib/python2.7/dist-packages/requests/sessions.py", line 573, in send
    r = adapter.send(request, **kwargs)
  File "/usr/local/lib/python2.7/dist-packages/requests/adapters.py", line 415, in send
    raise ConnectionError(err, request=request)
ConnectionError: ('Connection aborted.',

 365027487


TypeError: string indices must be integers, not str

In [10]:
len(controversy_stories)

13800

In [ ]:
 controversy_stories = cPickle.load(file( os.path.expanduser( '~/Dropbox/gamergate_data/controversy_stories.pickle' ), 'r') )

100 stories processed
752 names
[(u'Zoe Quinn', 11), (u'Anita Sarkeesian', 9), (u'Leigh Alexander', 8), (u'Adam Baldwin', 5), (u'Eron Gjoni', 4)]
200 stories processed
1155 names
[(u'Zoe Quinn', 23), (u'Anita Sarkeesian', 23), (u'Leigh Alexander', 10), (u'Brianna Wu', 9), (u'Adam Baldwin', 8)]
300 stories processed
1642 names
[(u'Anita Sarkeesian', 32), (u'Zoe Quinn', 30), (u'Brianna Wu', 15), (u'Leigh Alexander', 13), (u'Adam Baldwin', 9)]
400 stories processed
2128 names
[(u'Anita Sarkeesian', 44), (u'Zoe Quinn', 43), (u'Brianna Wu', 21), (u'Leigh Alexander', 15), (u'Adam Baldwin', 14)]
500 stories processed
2897 names
[(u'Anita Sarkeesian', 57), (u'Zoe Quinn', 54), (u'Brianna Wu', 27), (u'Leigh Alexander', 21), (u'Adam Baldwin', 14)]
600 stories processed
3299 names
[(u'Anita Sarkeesian', 69), (u'Zoe Quinn', 65), (u'Brianna Wu', 32), (u'Leigh Alexander', 23), (u'Adam Baldwin', 16)]
700 stories processed
3711 names
[(u'Anita Sarkeesian', 87), (u'Zoe Quinn', 77), (u'Brianna Wu', 41), 

In [ ]:
top_400_names = set([ n[0] for n in name_counts.most_common(400) ])

edge_counts = Counter()
stories_processed = 0

for stories_id in controversy_stories_corenlp.keys():
    if 'names' in controversy_stories_corenlp[ stories_id ]:

        story_top_names = [ name for name in controversy_stories_corenlp[ stories_id ] [ 'names' ]  if name in top_400_names]
        
        if len( story_top_names ) > 1 :
            #print story_top_names
            
            edges = sorted( list (itertools.combinations( story_top_names, 2 ) ) )

            edges = [ tuple( sorted(e) ) for e in edges ]

            for edge in edges:
                if edge not in edges:
                    edge_counts[edge] = 0

                edge_counts[ edge ] += 1
                
    stories_processed += 1
    
    if stories_processed % 1000 == 0:
        print stories_processed, "stories processed"
        print len( edge_counts.keys() ), "names"
        print edge_counts.most_common(5)
        
        #break


In [ ]:
if reprocess_with_cliff:
    for s in corenlp_stories:
        print s['stories_id']
        s['cliff_results'] = cliff_api.parseNlpJson( s['corenlp'])#['results']['people']
        del s['corenlp']

for edge in sorted(edge_counts.keys()):
    person_graph.add_edge( edge[0], edge[1], weight=edge_counts[edge])
    

len( list( nx.connected_components( person_graph) )[0] )
#person_graph.edges()
    
    

In [ ]:
controversy_stories = []
corenlp_stories = cPickle.load(file( os.path.expanduser( '~/Dropbox/gamergate_data/cliff_stories.pickle'), 'r') )

In [ ]:
names_stories_map = {}
stories_name_map = {}

for s in corenlp_stories:
    
    if 'cliff_results' not in s:
        print 'skipping story', s['stories_id']
        continue
        
    results = s['cliff_results']['results']
    
    if 'people' not in results:
        continue
        
    people = results['people']
    names = [ p['name'] for p in people ]
    
    stories_name_map[ s['stories_id']] = []
    
    for name in names:
        if name not in names_stories_map:
            names_stories_map[ name ] = []
            
        names_stories_map[name].append( s )
        
        stories_name_map[ s['stories_id']].append( name )
        

len( names_stories_map.keys())


In [ ]:
import itertools

edge_counts = Counter()

for names in stories_name_map.values():
    edges = list (itertools.combinations( names, 2 ) )
    
    for edge in edges:
        if edge not in edges:
            edge_counts[edge] = 0
    
        edge_counts[ edge ] += 1

edge_counts.most_common()    

In [ ]:
from collections import Counter

c = Counter (  dict( [ ( name, len ( names_stories_map[ name ] ) ) for name in names_stories_map.keys() ]  ) )
             

del c['Sepp Blatter']
del c['Ray Rice']

c.most_common()
